In [1]:
# | default_exp _application.tester

In [2]:
# | export

import asyncio
import collections
import inspect
from unittest.mock import AsyncMock, MagicMock
import json
from contextlib import asynccontextmanager
from itertools import groupby
from typing import *
from types import ModuleType

from pydantic import BaseModel

from fastkafka import KafkaEvent
from fastkafka._application.app import FastKafka, AwaitedMock, _get_kafka_brokers, _get_kafka_config
from fastkafka._components.asyncapi import KafkaBroker, KafkaBrokers
from fastkafka._components.helpers import unwrap_list_type
from fastkafka._components.meta import delegates, export, patch
from fastkafka._components.producer_decorator import unwrap_from_kafka_event
from fastkafka._components.aiokafka_consumer_loop import ConsumeCallable
from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker
from fastkafka._testing.in_memory_broker import InMemoryBroker
from fastkafka._testing.local_redpanda_broker import LocalRedpandaBroker
from fastkafka._components.helpers import remove_suffix

In [3]:
import pytest
from pydantic import Field

from fastkafka import EventMetadata, KafkaEvent
from fastkafka._components.logger import get_logger, suppress_timestamps

In [4]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [5]:
# | notest
# allows async calls in notebooks

import nest_asyncio

In [6]:
# | notest

nest_asyncio.apply()

In [7]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port="9092")))


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [8]:
# | export


def _get_broker_spec(bootstrap_server: str) -> KafkaBroker:
    """
    Helper function to get the broker specification from the bootstrap server URL.

    Args:
        bootstrap_server: The bootstrap server URL in the format "<host>:<port>".

    Returns:
        A KafkaBroker object representing the broker specification.
    """
    url = bootstrap_server.split(":")[0]
    port = bootstrap_server.split(":")[1]
    return KafkaBroker(url=url, port=port, description="", protocol="")

## Fastkafka Tester class

In [9]:
# | export


@export("fastkafka.testing")
class Tester(FastKafka):
    __test__ = False

    def __init__(
        self,
        app: Union[FastKafka, List[FastKafka]],
        *,
        use_in_memory_broker: bool = True,
        **kwargs: Any,
    ):
        """Mirror-like object for testing a FastKafka application

        Can be used as context manager

        Args:
            app: The FastKafka application to be tested.
            use_in_memory_broker: Whether to use an in-memory broker for testing or not.
        """
        self.apps = app if isinstance(app, list) else [app]

        for app in self.apps:
            app.create_mocks()

        super().__init__()
        self.mirrors: Dict[Any, Any] = {}
        self.apps_initial_brokers: Dict[FastKafka, Any] = dict()
        self.use_external_broker: bool = False
        
        self._kafka_brokers = self.apps[0]._kafka_brokers
        self._kafka_config["bootstrap_servers_id"] = self.apps[0]._kafka_config[
            "bootstrap_servers_id"
        ]
            
        self.use_in_memory_broker = use_in_memory_broker

    async def _start_tester(self) -> None:
        """Starts the Tester"""
        for app in self.apps:
            await app.__aenter__()
        self.create_mocks()
        self._arrange_mirrors()
        await super().__aenter__()
        await asyncio.sleep(3)

    async def _stop_tester(self) -> None:
        """Shuts down the Tester"""
        await super().__aexit__(None, None, None)
        for app in self.apps[::-1]:
            await app.__aexit__(None, None, None)

    def _create_mirrors(self) -> None:
        pass
    
    def _clean_mirrors(self) -> None:
        pass

    def _arrange_mirrors(self) -> None:
        pass
    
    @delegates(_get_kafka_config)
    def using_external_broker(self, 
                              broker: Optional[Dict[str, Any]],
                              **kwargs: Any,
                             ) -> "Tester":
        
        self.use_external_broker = True
        self._kafka_brokers = _get_kafka_brokers(broker)
        self._kafka_config["bootstrap_servers_id"] = _get_kafka_config(**kwargs)[
            "bootstrap_servers_id"
        ]    
        
        for app in self.apps:
            # make a copy of apps kafka_brokers and bootstrap_servers_id
            self.apps_initial_brokers[app] = dict(
                _kafka_brokers = app._kafka_brokers.model_copy(),
                bootstrap_servers_id = app._kafka_config["bootstrap_servers_id"]
            )

            app._kafka_brokers = self._kafka_brokers
            app._kafka_config["bootstrap_servers_id"] = self._kafka_config["bootstrap_servers_id"]  
            
        return self

    @asynccontextmanager
    async def _create_ctx(self) -> AsyncGenerator["Tester", None]:
        self._create_mirrors()
        
        if self.use_in_memory_broker == True and self.use_external_broker == False:
            with InMemoryBroker(): # type: ignore
                await self._start_tester()
                try:
                    yield self
                finally:
                    await self._stop_tester()
        else:
            await self._start_tester()
            try:
                yield self
            finally:
                await self._stop_tester()
                if self.use_external_broker:
                    # for each app, set back the initial kafka_brokers and bootstrap_servers_id
                    for app in self.apps:
                        app._kafka_brokers = self.apps_initial_brokers[app]["_kafka_brokers"]
                        app._kafka_config["bootstrap_servers_id"] = self.apps_initial_brokers[app]["bootstrap_servers_id"]
                        
        self._clean_mirrors()
                        

    async def __aenter__(self) -> "Tester":
        self._ctx = self._create_ctx()
        return await self._ctx.__aenter__()

    async def __aexit__(self, *args: Any) -> None:
        await self._ctx.__aexit__(*args)

In [10]:
for _ in range(2):
    with pytest.raises(RuntimeError) as e:
        async with Tester(app) as tester:
            assert tester.is_started
            raise RuntimeError("ok")

    print(e)
    assert not tester.is_started

<function on_preprocessed_signals at 0x7efde9451da0>
<function to_predictions at 0x7efde9451b20>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer

In [11]:
tester = Tester(app)

@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

<function on_preprocessed_signals at 0x7efde92a8360>
<function to_predictions at 0x7efde92a8680>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function on_predictions at 0x7efe04bdeca0>
<function to_preprocessed_signals at 0x7efde9452980>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Cons

## Test multiple brokers

In [12]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port="9092")])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port="9092"))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals")
async def on_preprocessed_signals_1(msg: TestMsg):
    print(f"Default broker:  {msg=}")
    
@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals_2(msg: TestMsg):
    print(f"Defined broker:  {msg=}")

tester = Tester(app)


@tester.produces(brokers=kafka_brokers_2)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals_2.assert_called(
        timeout=5
    )
    await app.awaited_mocks.on_preprocessed_signals_1.assert_not_called(
        timeout=5
    )

print("ok")

<function on_preprocessed_signals_1 at 0x7efde9311f80>
<function on_preprocessed_signals_2 at 0x7efde9311ee0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
<function to_preprocessed_signals at 0x7efe0024cf40>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': ['server_1:9092']}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._com

In [13]:
kafka_brokers_1 = dict(localhost=dict(url="server_1", port="9092"))
kafka_brokers_2 = dict(localhost=[dict(url="server_2", port="9092")])

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


tester = Tester(app)


@tester.produces(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


@tester.consumes(auto_offset_reset="earliest", brokers=kafka_brokers_1)
async def on_predictions(msg: TestMsg):
    print(f"tester: {msg=}")


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

<function on_preprocessed_signals at 0x7efde92a8860>
<function to_predictions at 0x7efde92a84a0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_1:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function on_predictions at 0x7efde97309a0>
<function to_preprocessed_signals at 0x7efde93dc0e0>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Cons

## Mirroring

In [14]:
# | export


def mirror_producer(
    topic: str, producer_f: Callable[..., Any], brokers: str, app: FastKafka
) -> Callable[..., Any]:
    """
    Decorator to create a mirrored producer function.

    Args:
        topic: The topic to produce to.
        producer_f: The original producer function.
        brokers: The brokers configuration.
        app: The FastKafka application.

    Returns:
        The mirrored producer function.
    """
    msg_type = inspect.signature(producer_f).return_annotation

    msg_type_unwrapped = unwrap_list_type(unwrap_from_kafka_event(msg_type))

    async def skeleton_func(msg: BaseModel) -> None:
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name, take into consideration the origin app and brokers
    # configuration so that we can differentiate those two
    mirror_func.__name__ = f"mirror_{id(app)}_on_{remove_suffix(topic).replace('.', '_')}_{abs(hash(brokers))}"

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type_unwrapped,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig  # type: ignore

    return mirror_func

In [15]:
app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> TestMsg:
    pass


@app.produces(topic="topic2", brokers=dict(localhost=dict(url="localhost", port=9093)))
async def some_log_1(in_var: int) -> TestMsg:
    pass


@app.produces(topic="topic2", brokers=dict(localhost=dict(url="localhost", port=9093)))
async def some_log_2(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, brokers, _) in app._producers_store.items():
    mirror = mirror_producer(
        topic,
        producer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app,
    )
    assert "_".join(mirror.__name__.split("_")[2:-1]) == "on_" + remove_suffix(topic)
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [16]:
app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))

@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> KafkaEvent[List[TestMsg]]:
    pass


for topic, (producer_f, _, brokers, _) in app._producers_store.items():
    mirror = mirror_producer(
        topic,
        producer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app
    )
    assert "_".join(mirror.__name__.split("_")[2:-1]) == "on_" + remove_suffix(topic)
    assert inspect.signature(mirror).parameters["msg"].annotation.__name__ == "TestMsg"

In [17]:
# | export


def mirror_consumer(
    topic: str, consumer_f: Callable[..., Any], brokers: str, app: FastKafka
) -> Callable[[BaseModel], Coroutine[Any, Any, BaseModel]]:
    """
    Decorator to create a mirrored consumer function.

    Args:
        topic: The topic to consume from.
        consumer_f: The original consumer function.
        brokers: The brokers configuration.
        app: The FastKafka application.

    Returns:
        The mirrored consumer function.
    """
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    msg_type_unwrapped = unwrap_list_type(msg_type)

    async def skeleton_func(msg: BaseModel) -> BaseModel:
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name, take into consideration the origin app and brokers
    # configuration so that we can differentiate those two
    mirror_func.__name__ = f"mirror_{id(app)}_to_{remove_suffix(topic).replace('.', '_')}_{abs(hash(brokers))}"

    # adjust arg and return val
    sig = sig.replace(
        parameters=[msg_type], return_annotation=msg_type_unwrapped.annotation
    )

    mirror_func.__signature__ = sig  # type: ignore
    return mirror_func

In [18]:
for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
    mirror = mirror_consumer(
        topic,
        consumer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app
    )
    assert "_".join(mirror.__name__.split("_")[3:-1]) == "to_" + remove_suffix(topic)
    assert (
        inspect.signature(mirror).return_annotation.__name__ == TestMsg.__name__
    ), inspect.signature(mirror).return_annotation.__name__
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [19]:
# | export


@patch
def _create_mirrors(self: Tester) -> None:
    """
    Creates mirror functions for producers and consumers.

    Iterates over the FastKafka application and its producers and consumers. For each consumer, it creates a mirror
    consumer function using the `mirror_consumer` decorator. For each producer, it creates a mirror producer function
    using the `mirror_producer` decorator. The mirror functions are stored in the `self.mirrors` dictionary and also
    set as attributes on the Tester instance.

    Returns:
        None
    """
    for app in self.apps:
        for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(
                topic,
                consumer_f,
                brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
                app,
            )
            mirror_f = self.produces(  # type: ignore
                topic=remove_suffix(topic),
                brokers=brokers,
            )(mirror_f)
            self.mirrors[consumer_f] = mirror_f
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, brokers, _) in app._producers_store.items():
            mirror_f = mirror_producer(
                topic,
                producer_f,
                brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
                app,
            )
            mirror_f = self.consumes(
                topic=remove_suffix(topic),
                brokers=brokers,
            )(
                mirror_f  # type: ignore
            )
            self.mirrors[producer_f] = mirror_f
            setattr(self, mirror_f.__name__, mirror_f)

In [20]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))
app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

async with Tester(app) as tester:
    assert hasattr(
        tester,
        f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
    )

    assert hasattr(
        tester,
        f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(app._kafka_brokers).model_dump_json()))}",
    )

<function on_preprocessed_signals at 0x7efe043d98a0>
<function to_predictions at 0x7efde93cafc0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_1:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde9371b20>
<function mirror_consumer.<locals>.skeleton_func at 0x7efe04bdeca0>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consum

In [21]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester(app) as tester:
    await getattr(
        tester,
        f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
    )(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await getattr(
        tester.awaited_mocks,
        f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(kafka_brokers_1).model_dump_json()))}",
    ).assert_called(timeout=5)

print("ok")

<function on_preprocessed_signals at 0x7efde9450a40>
<function to_predictions at 0x7efde9451ee0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_1:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde92efba0>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde9730d60>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consum

In [22]:
# | export


@patch
def _clean_mirrors(self: Tester) -> None:  
    for mirror_f in set(self.mirrors.values()):
        if hasattr(mirror_f, '__call__'):
            delattr(self, mirror_f.__name__)
        
    self._consumers_store = {}
    self._producers_store = {}
    self.mirrors = {}

In [23]:
tester = Tester([app])

async with tester:
    assert hasattr(
        tester,
        f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
    )
    assert hasattr(
        tester,
        f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(app._kafka_brokers).model_dump_json()))}",
    )

assert not hasattr(
        tester,
        f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
    )
assert not hasattr(
    tester,
    f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(app._kafka_brokers).model_dump_json()))}",
)

<function on_preprocessed_signals at 0x7efde92a91c0>
<function to_predictions at 0x7efde92ef740>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_1:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde9453ec0>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde9730d60>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consum

## Mirrors dict and syntax sugar

In [24]:
# | export


class AmbiguousWarning:
    """
    Warning class used for ambiguous topics.

    Args:
        topic: The ambiguous topic.
        functions: List of function names associated with the ambiguous topic.
    """
    def __init__(self, topic: str, functions: List[str]):
        self.topic = topic
        self.functions = functions

    def __getattribute__(self, attr: str) -> Any:
        raise RuntimeError(
            f"Ambiguous topic: {super().__getattribute__('topic')}, for functions: {super().__getattribute__('functions')}\nUse Tester.mirrors[app.function] to resolve ambiguity"
        )

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        raise RuntimeError(
            f"Ambiguous topic: {self.topic}, for functions: {self.functions}\nUse Tester.mirrors[app.function] to resolve ambiguity"
        )

In [25]:
with pytest.raises(Exception) as e:
    AmbiguousWarning(topic="some_topic", functions=["some_functions"])(TestMsg(msg="signal"))
    
assert e.value.args[0] == "Ambiguous topic: some_topic, for functions: ['some_functions']\nUse Tester.mirrors[app.function] to resolve ambiguity"

with pytest.raises(Exception) as e:
    AmbiguousWarning(topic="some_topic", functions=["some_brokers"]).assert_called(timeout=5)
    
assert e.value.args[0] == "Ambiguous topic: some_topic, for functions: ['some_brokers']\nUse Tester.mirrors[app.function] to resolve ambiguity"

In [26]:
# | export


def set_sugar(
    *,
    tester: Tester,
    prefix: str,
    topic_brokers: Dict[str, Tuple[List[str], List[str]]],
    topic: str,
    brokers: str,
    origin_function_name: str,
    function: Callable[..., Union[Any, Awaitable[Any]]],
) -> None:
    """
    Sets the sugar function for a topic.

    Args:
        tester: The Tester instance.
        prefix: The prefix to use for the sugar function (e.g., "to_" or "on_").
        topic_brokers: Dictionary to store the brokers and functions associated with each topic.
        topic: The topic name.
        brokers: The brokers configuration.
        origin_function_name: The name of the original function.
        function: The mirror function to be set as the sugar function.

    Returns:
        None
    """
    brokers_for_topic, functions_for_topic = topic_brokers.get(topic, ([], []))
    if brokers not in brokers_for_topic:
        brokers_for_topic.append(brokers)
        functions_for_topic.append(origin_function_name)
        topic_brokers[topic] = (brokers_for_topic, functions_for_topic)
    if len(brokers_for_topic) == 1:
        setattr(tester, f"{prefix}{topic}", function)
    else:
        setattr(
            tester, f"{prefix}{topic}", AmbiguousWarning(topic, functions_for_topic)
        )

In [41]:
# | export


@patch
def _arrange_mirrors(self: Tester) -> None:
    """
    Arranges the mirror functions.

    Iterates over the FastKafka application and its producers and consumers. For each consumer, it retrieves the mirror
    function from the `self.mirrors` dictionary and sets it as an attribute on the Tester instance. It also sets the
    sugar function using the `set_sugar` function. For each producer, it retrieves the mirror function and sets it as
    an attribute on the Tester instance. It also sets the sugar function for the awaited mocks. Finally, it creates the
    `mocks` and `awaited_mocks` namedtuples and sets them as attributes on the Tester instance.

    Returns:
        None
    """
    topic_brokers: Dict[str, Tuple[List[str], List[str]]] = {}
    mocks = {}
    awaited_mocks = {}
    
    for app in self.apps:
        for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
            mirror_f = self.mirrors[consumer_f]
            self.mirrors[getattr(app, consumer_f.__name__)] = mirror_f
            set_sugar(
                tester=self,
                prefix="to_",
                topic_brokers=topic_brokers,
                topic=remove_suffix(topic),
                brokers=brokers.model_dump_json()
                if brokers is not None
                else app._kafka_brokers.model_dump_json(),
                origin_function_name=consumer_f.__name__,
                function=mirror_f,
            )

            mocks[f"to_{remove_suffix(topic)}"] = getattr(self.mocks, mirror_f.__name__)
            awaited_mocks[f"to_{remove_suffix(topic)}"] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )

        for topic, (producer_f, _, brokers, _) in app._producers_store.items():
            mirror_f = self.mirrors[producer_f]
            self.mirrors[getattr(app, producer_f.__name__)] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )
            set_sugar(
                tester=self,
                prefix="on_",
                topic_brokers=topic_brokers,
                topic=remove_suffix(topic),
                brokers=brokers.model_dump_json()
                if brokers is not None
                else app._kafka_brokers.model_dump_json(),
                origin_function_name=producer_f.__name__,
                function=getattr(self.awaited_mocks, mirror_f.__name__),
            )
            mocks[f"on_{remove_suffix(topic)}"] = getattr(self.mocks, mirror_f.__name__)
            awaited_mocks[f"on_{remove_suffix(topic)}"] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )

    AppMocks = collections.namedtuple(  # type: ignore
        f"{self.__class__.__name__}Mocks", [f_name for f_name in mocks]
    )
    setattr(self, "mocks", AppMocks(**mocks))
    setattr(self, "awaited_mocks", AppMocks(**awaited_mocks))

In [43]:
# Test batch mirroring


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg, meta: EventMetadata):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]


async with Tester(second_app) as tester:
    assert hasattr(
        tester,
        f"mirror_{id(second_app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(second_app._kafka_brokers).model_dump_json()))}",
    )    

    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
    tester.mocks.on_predictions.assert_called()
print("ok")

<function on_preprocessed_signals at 0x7efde9140e00>
<function to_predictions at 0x7efde91407c0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde8f3fb00>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde8f13f60>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consum

In [29]:
kafka_brokers_1 = dict(localhost=dict(url="server_1", port=9092))
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals")
async def on_preprocessed_signals_1(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals_2(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


async with Tester(app) as tester:
    with pytest.raises(Exception) as exception_produce:
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    assert (
        exception_produce.value.args[0]
        == "Ambiguous topic: preprocessed_signals, for functions: ['on_preprocessed_signals_1', 'on_preprocessed_signals_2']\nUse Tester.mirrors[app.function] to resolve ambiguity"
    )
    await tester.mirrors[on_preprocessed_signals_1](TestMsg(msg="signal"))

<function on_preprocessed_signals_1 at 0x7efde9310540>
<function on_preprocessed_signals_2 at 0x7efde9313ba0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
<function mirror_consumer.<locals>.skeleton_func at 0x7efde9310900>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde9453380>
Before arange mirrors
 for key <function on_preprocessed_signals_1 at 0x7efde9452d40> attr: mirror_139629004321552_to_preprocessed_signals_3002834353504572060
 for key <function on_preprocessed_signals_2 at 0x7efde9451da0> attr: mirror_139629004321552_to_preprocessed_signals_146123504123104211
consumer mirror = <function on_preprocessed_signals_1 at 0x7efde9310540>  -  <function mirror_consumer.<locals>.skeleton_func at 0x7efde94519e0>
consumer mirror = <function on_preprocessed_signals_2 at 0x7efde9313ba0>  -  <function mirror_consumer.<locals>.skeleto

In [30]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_1 = dict(localhost=[dict(url="server_2", port=9092)])

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.produces(topic="predictions")
async def to_predictions_1(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


@app.produces(topic="predictions", brokers=kafka_brokers_2)
async def to_predictions_2(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester(app) as tester:
    with pytest.raises(Exception) as exception_consume:
        await tester.on_predictions.assert_called(timeout=5)
    assert (
        exception_consume.value.args[0]
        == "Ambiguous topic: predictions, for functions: ['to_predictions_1', 'to_predictions_2']\nUse Tester.mirrors[app.function] to resolve ambiguity"
    )
    await tester.mirrors[app.to_predictions_1].assert_not_called(timeout=5)

<function to_predictions_1 at 0x7efde9370ae0>
<function to_predictions_2 at 0x7efde9370b80>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde94519e0>
<function mirror_producer.<locals>.skeleton_func at 0x7efde9371260>
Before arange mirrors
 for key <function to_predictions_1 at 0x7efde9453d80> attr: mirror_139629006470096_on_predictions_772605455916566904

In [31]:
# Test KafkaEvent mirroring and consumer batching


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: List[TestMsg]):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> KafkaEvent[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return KafkaEvent(message=prediction, key=b"123")


async with Tester(second_app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.on_predictions.assert_called(timeout=5)
print("ok")

<function on_preprocessed_signals at 0x7efde91d4400>
<function to_predictions at 0x7efde91d5c60>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde92a82c0>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde91d4860>
Before arange mirrors
 for key <function on_preprocessed_signals at 0x7efde91d6340> attr: mirror_139629005479120_to_preprocessed_signals_4468926502393708864
 for key <function to_predictions at 0x7efde92a9da0> attr: mirror_139629005479120_on_predictions_4468926502393708864
consumer mirror = <function on_preprocessed_signals at 0x7efde91d4400>  -  <fun

In [32]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="server_2", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.mirrors[second_app.to_predictions].assert_called(timeout=5)
print("ok")

<function to_predictions_1 at 0x7efde9453d80>
<function to_predictions_2 at 0x7efde9452660>
<function on_preprocessed_signals at 0x7efde9372520>
<function to_predictions at 0x7efde91d60c0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AI

In [33]:
def get_server_name(broker: Optional[Dict[str, Any]]) -> str:
    return broker['url'] if broker else None

In [34]:
get_server_name(None)

In [35]:
kafka_brokers_1 = dict(localhost=dict(url="server", port=9092), production=dict(url="prod_server", port=9092))
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092), production=dict(url="prod_server2", port=9092))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.produces(topic="predictions")
async def to_predictions_1(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


@app.produces(topic="predictions", brokers=kafka_brokers_2)
async def to_predictions_2(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [36]:
class TestMsg(BaseModel):
    msg: str = Field(...)


kafka_brokers = dict(localhost=dict(url="server_3", port=9092), production=dict(url="prod_server", port=9092))
#kafka_brokers_1 = dict(localhost=[dict(url="server", port=9092)], production=dict(url="prod_server", port=9092))
second_app = FastKafka(kafka_brokers=kafka_brokers, bootstrap_servers_id="production")

#second_app = FastKafka(kafka_brokers=kafka_brokers)
tester_broker = dict(localhost=dict(url="server__tester", port=9092), production=dict(url="prod_server2", port=9092))
#kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

with InMemoryBroker() as broker:
#     async with Tester([second_app, app], use_in_memory_broker=False, broker=tester_broker, bootstrap_servers_id="production") as tester:
    async with Tester([second_app, app], use_in_memory_broker=False).using_external_broker(broker=tester_broker, bootstrap_servers_id="production") as tester:
        assert tester.use_external_broker == True
        assert tester._kafka_brokers == second_app._kafka_brokers
        #assert tester._kafka_brokers == app._kafka_brokers
        assert second_app._kafka_brokers != _get_kafka_brokers(kafka_brokers)
        assert second_app._kafka_brokers == _get_kafka_brokers(tester_broker)
        
        assert tester._kafka_config["bootstrap_servers_id"] == second_app._kafka_config["bootstrap_servers_id"]
        #assert tester._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]
        assert tester._kafka_config["bootstrap_servers_id"] == _get_kafka_config(bootstrap_servers_id="production")[
                "bootstrap_servers_id"
        ]    
        
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await tester.mirrors[second_app.to_predictions].assert_called(timeout=5)
    print("ok")

# assert _kafka_brokers were returned to the initial ones
assert second_app._kafka_brokers == _get_kafka_brokers(kafka_brokers)
assert second_app._kafka_brokers != _get_kafka_brokers(tester_broker)

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
<function on_preprocessed_signals at 0x7efde92044a0>
<function to_predictions at 0x7efde92054e0>
<function to_predictions_1 at 0x7efde9205260>
<function to_predictions_2 at 0x7efde9204b80>
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'prod_server2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'prod_server2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'prod_server2:9092'}'
[INFO] fastkafka._testing.in_memory_

In [37]:
tester = Tester([second_app])

with InMemoryBroker() as broker:
    async with tester:
        pass
    
with InMemoryBroker() as broker:
    async with tester:
        pass

<function on_preprocessed_signals at 0x7efde9373c40>
<function to_predictions at 0x7efde92a8540>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'prod_server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde9227c40>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde9227ec0>
Before arange mirrors
 for key <function on_preprocessed_signals at 0x7efde92247c0> attr: mirror_139629003105488_to_preprocessed_signals_3693427331988111246

In [39]:
tester = Tester([second_app], use_in_memory_broker=False).using_external_broker(broker=tester_broker, bootstrap_servers_id="production")

with InMemoryBroker() as broker:
    async with tester:
        print(tester._consumers_store.values())
        assert tester._kafka_config["bootstrap_servers_id"] == second_app._kafka_config["bootstrap_servers_id"]
        #assert tester._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]
        assert tester._kafka_config["bootstrap_servers_id"] == _get_kafka_config(bootstrap_servers_id="production")[
                "bootstrap_servers_id"
        ]    
        
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await tester.mirrors[second_app.to_predictions].assert_called(timeout=5)
    print("ok")
    
tester = tester.using_external_broker(broker=tester_broker, bootstrap_servers_id="production")
#tester._create_mirrors()
print(tester._consumers_store.values())
with InMemoryBroker() as broker:
    async with tester:
        print(tester._consumers_store.values())
        assert tester._kafka_config["bootstrap_servers_id"] == second_app._kafka_config["bootstrap_servers_id"]
        #assert tester._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]
        assert tester._kafka_config["bootstrap_servers_id"] == _get_kafka_config(bootstrap_servers_id="production")[
                "bootstrap_servers_id"
        ]    
        
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await tester.mirrors[second_app.to_predictions].assert_called(timeout=5)
    print("ok")
    

<function on_preprocessed_signals at 0x7efde92244a0>
<function to_predictions at 0x7efde92245e0>
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'prod_server2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
<function mirror_producer.<locals>.skeleton_func at 0x7efde92ef6a0>
<function mirror_consumer.<locals>.skeleton_func at 0x7efde94528e0>
Before arange mirrors
 for key <function on_preprocessed_signals at 0x7efde92ef600> attr: mirror_139629003105488_to_preprocessed_signals_5854846907140063990
 for key <function to_predictions at 0x7efde94518a0> attr: mirror_139629003105488_on_predictions_5854846907140063990
consumer mirror = <function on_preprocessed_signals at 0x7efde92044a0>  -  <

['AppMocks',
 '__aenter__',
 '__aexit__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__test__',
 '__weakref__',
 '_arrange_mirrors',
 '_asyncapi_path',
 '_bg_task_group_generator',
 '_bg_tasks_group',
 '_clean_mirrors',
 '_consumers_store',
 '_contact_info',
 '_create_ctx',
 '_create_mirrors',
 '_ctx',
 '_description',
 '_is_shutting_down',
 '_is_started',
 '_kafka_brokers',
 '_kafka_config',
 '_kafka_consumer_tasks',
 '_kafka_producer_tasks',
 '_kafka_service_info',
 '_on_error_topic',
 '_override_brokers',
 '_populate_bg_tasks',
 '_populate_consumers',
 '_populate_producers',
 '_producers_list',
 '_producers_store',
 '_root_path',
 '_running_bg_tasks',
 '_sche